1. Read the documentation of Language modelling in the Transformers library.
2. Download three Polish models from the Huggingface repository.

In [17]:
import pandas as pd
from transformers import pipeline

fill_mask_model_1 = pipeline(task="fill-mask", model='allegro/herbert-base-cased')
# fill_mask_model_2 = pipeline(task="fill-mask", model='allegro/herbert-base-cased')
# fill_mask_model_3 = pipeline(task="fill-mask", model='allegro/herbert-base-cased')


In [102]:
def parse_preds(preds):
    return [
        {
            "score": round(pred["score"], 3),
            "token": pred["token"],
            "token_str": pred["token_str"],
            "sequence": pred["sequence"],
        }
        for pred in preds
    ]
def parse_pred(pred, name):
    return {
                "model": name,
                "score": round(pred["score"], 3),
                "token": pred["token"],
                "token_str": pred["token_str"],
                "sequence": pred["sequence"],
            }

def preds_to_df(parsed_preds_list):
    return pd.DataFrame(*parsed_preds_list, index='model')


3. Devise a method to test if the langage model understands Polish cases. E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]", and the masked word should be in nominative case. Create sentences for each case.

In [103]:
text =  "Warszawa to największe <mask>"
top_k = 1
preds = fill_mask_model_1(text, top_k=1)
k=0
preds_to_df([parse_pred(preds[k], 'bert'), parse_pred(preds[k], 'bert2'), parse_pred(preds[k], 'bert3')])
    

TypeError: DataFrame.__init__() got multiple values for argument 'index'

4. Devise a method to test long-range relationships such as gender. E.e. you can use two verbs where withe masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [19]:
text =  "Byłam w sklepie, kupiłam chleb i <mask> do domu"
preds = fill_mask_model_1(text, top_k=3)

parse_preds(preds)

[{'score': 0.0994,
  'token': 22636,
  'token_str': 'wracam',
  'sequence': 'Byłam w sklepie, kupiłam chleb i wracam do domu'},
 {'score': 0.043,
  'token': 19464,
  'token_str': 'poszła',
  'sequence': 'Byłam w sklepie, kupiłam chleb i poszła do domu'},
 {'score': 0.0425,
  'token': 35614,
  'token_str': 'wróciłem',
  'sequence': 'Byłam w sklepie, kupiłam chleb i wróciłem do domu'}]

5. Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [21]:
text =  "<mask> wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza."
preds = fill_mask_model_1(text, top_k=1)

parse_preds(preds)

[{'score': 0.362,
  'token': 17550,
  'token_str': 'Woda',
  'sequence': 'Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.'}]

In [24]:
text =  "Najwyższą <mask> na świecie jest Mount Everest"
preds = fill_mask_model_1(text, top_k=1)

parse_preds(preds)

[{'score': 0.8841,
  'token': 22591,
  'token_str': 'górą',
  'sequence': 'Najwyższą górą na świecie jest Mount Everest'}]

In [29]:
text =  "Na Księżycu przyśpieszenie grawitacyjne jest <mask> niż na Ziemii"
preds = fill_mask_model_1(text, top_k=3)

parse_preds(preds)

[{'score': 0.3849,
  'token': 5185,
  'token_str': 'większe',
  'sequence': 'Na Księżycu przyśpieszenie grawitacyjne jest większe niż na Ziemii'},
 {'score': 0.2154,
  'token': 8921,
  'token_str': 'wyższe',
  'sequence': 'Na Księżycu przyśpieszenie grawitacyjne jest wyższe niż na Ziemii'},
 {'score': 0.1163,
  'token': 9662,
  'token_str': 'mniejsze',
  'sequence': 'Na Księżycu przyśpieszenie grawitacyjne jest mniejsze niż na Ziemii'}]

6. Take into accout the fact, that causal language models such as PapuGaPT2 or plT5, will only generate continuations of the sentenes, so the examples have to be created according to that paradigm.
